In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import time
import math
from sklearn.model_selection import KFold
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from tqdm import tqdm
from torch.optim import lr_scheduler

base_to_int = {'A': 0, 'C': 1, 'G': 2, 'U': 3, 'PAD': 4}

/home/mah20012/envs/cse1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# takes some time
df = pd.read_csv('../train_data.csv')

df_2A3 = df[df['experiment_type'] == '2A3_MaP']
df_DMS = df[df['experiment_type'] == 'DMS_MaP']
print(len(df_2A3))
print(len(df_DMS))

821840
821840


In [3]:
# torch.cuda.device = 1

In [4]:
# df_2A3 = df_2A3.sample(n=int(len(df_2A3)/4), replace=False)
df_2A3 = df_2A3.sample(n=15000, replace=False)
df_DMS = df_DMS.sample(n=15000, replace=False)
print(len(df_2A3))
print(len(df_DMS))

15000
15000


In [5]:
class RNADataset(Dataset):
    def __init__(self, dataframe, max_length, context_size=3):
        self.dataframe = dataframe
        self.max_length = max_length
        self.context_size = context_size
        self.pad_id = base_to_int['PAD']
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        sequence = self.dataframe.iloc[idx, 1]

        padded_sequence = self.encode_seq(sequence)
        
        # reactivity_0001 -> reactivity_0206
        reactivity = self.dataframe.iloc[idx, 6:212].fillna(0).values.astype(float)
        reactivity_padded = list(reactivity) + [0]*(self.max_length - len(reactivity))
        reactivity_padded = reactivity_padded[:self.max_length]
        
        mask = [1 if i < len(padded_sequence) else 0 for i in range(self.max_length)]
        
        return padded_sequence, torch.tensor(reactivity_padded, dtype=torch.float), torch.tensor(mask, dtype=torch.float)
        
    
    def encode_seq(self, sequence):
        # Convert sequence to list of integers using the dictionary
        encoded_sequence = [base_to_int[base] for base in sequence]
        
        # Pad the sequence to the specified max_length
        padded_sequence = encoded_sequence + [base_to_int['PAD']] * (self.max_length - len(encoded_sequence))
        padded_sequence = padded_sequence[:self.max_length]

        return torch.tensor(padded_sequence, dtype=torch.long)


In [6]:
max_length = max(df_2A3['sequence'].apply(len))
dataset = RNADataset(df_2A3, max_length)

In [7]:
dataset[0]

(tensor([2, 2, 2, 0, 0, 1, 2, 0, 1, 3, 1, 2, 0, 2, 3, 0, 2, 0, 2, 3, 1, 2, 0, 0,
         0, 0, 3, 1, 3, 3, 0, 3, 3, 0, 3, 3, 1, 0, 3, 3, 2, 3, 3, 0, 0, 3, 2, 1,
         1, 3, 0, 3, 1, 3, 3, 0, 0, 1, 1, 3, 3, 2, 0, 1, 1, 0, 2, 2, 2, 1, 3, 3,
         3, 0, 0, 1, 3, 2, 1, 0, 2, 0, 2, 3, 1, 0, 1, 0, 3, 2, 3, 3, 2, 0, 1, 0,
         1, 3, 2, 0, 1, 3, 3, 0, 0, 1, 0, 0, 0, 2, 1, 1, 3, 3, 0, 1, 0, 3, 3, 0,
         0, 2, 3, 2, 2, 2, 2, 0, 1, 0, 1, 3, 2, 3, 3, 1, 1, 0, 1, 3, 3, 1, 2, 2,
         3, 2, 2, 0, 0, 1, 0, 2, 3, 2, 3, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 1,
         0, 0, 1, 0, 0, 1, 0, 0, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]),
 tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0

In [8]:
padded_sequence, reactivity_padded, mask_padded = dataset[0]
padded_sequence.shape, reactivity_padded.shape, mask_padded.shape

(torch.Size([206]), torch.Size([206]), torch.Size([206]))

In [9]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.6, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [10]:
class TransformerModel(nn.Module):
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.6):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(ninp, dropout)

        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)

        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, 1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output.squeeze(-1)


In [11]:
class MaskedHuberLoss(nn.Module):
    def __init__(self, delta=1.0):
        super(MaskedHuberLoss, self).__init__()
        self.delta = delta

    def forward(self, input, target, mask):
        clipped_target = torch.clip(target, min=0, max=1)

        loss = torch.where(torch.abs(input - clipped_target) < self.delta,
                           0.5 * (input - clipped_target) ** 2,
                           self.delta * (torch.abs(input - clipped_target) - 0.5 * self.delta))
        loss = loss * mask
        return loss.sum() / mask.sum()


In [12]:
ntokens = len(base_to_int)  # Vocabulary size
emsize = 300  # Increase embedding dimensions to 300
nhid = 300    # Increase hidden layer size to 300
nlayers = 2   # Increase the number of layers in the transformer to 6
nhead = 100   # Increase the number of heads in multi-head attention to 6
dropout = 0.6 # Increase dropout value to 0.6
weight_decay = 0.0001

# Data loading and preprocessing
max_length = max(df_2A3['sequence'].apply(len))
dataset = RNADataset(df_2A3, max_length)

# Setting up cross-validation
kf = KFold(n_splits=3)

In [13]:
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
for sequences, reactivities, masks in dataloader:
    print(sequences.shape, reactivities.shape, masks.shape)
    break


torch.Size([128, 206]) torch.Size([128, 206]) torch.Size([128, 206])


In [14]:
# device = 

In [15]:
torch.cuda.device_count()

8

In [22]:
device = torch.device("cuda:1")

In [23]:
for fold, (train_index, test_index) in enumerate(kf.split(dataset)):
    print(f"Fold {fold + 1}")
    train_dataset = torch.utils.data.Subset(dataset, train_index)
    test_dataset = torch.utils.data.Subset(dataset, test_index)
    
    train_dataloader = DataLoader(train_dataset, batch_size=160, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=160, shuffle=False)

    # Initialize the model and loss function
    model1 = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
    loss_fn = MaskedHuberLoss()
    optimizer = optim.Adam(model1.parameters(), weight_decay=0.0001, lr=5e-3)
    
    # Setting up the scheduler
    def lambda_epoch(epoch):
        max_epoch = 3
        return math.pow((1-epoch/max_epoch), 0.9)

    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch)

    # Training loop
    num_epochs = 3
    
    for epoch in range(num_epochs):
        start_time = time.time()
        model1.train()
        total_loss = 0.0
        for batch, (seq, target, mask) in enumerate(train_dataloader):
            seq = seq.to(device)
            target = target.to(device)
            mask = mask.to(device)
            
            optimizer.zero_grad()
            output = model1(seq)
            loss = loss_fn(output, target, mask)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            if batch % 300 == 0 and batch > 0:
                current = batch * len(seq)
                total = len(train_dataloader.dataset)
                elapsed = time.time() 
                loss_per_batch = total_loss / batch
                print(f'| Epoch {epoch+1:3d}/{num_epochs:3d} | {current:5d}/{total:5d} sequences | Mean Loss Batch: {loss_per_batch} | Elapsed time {elapsed - start_time:.2f}s')
        
        # Execute the learning rate scheduler once
        scheduler.step()
    
        elapsed_time = time.time() - start_time
        print(f'Epoch {epoch+1:3d} completed in {elapsed_time:.2f}s, Total loss {total_loss:.4f}')
            
    torch.save(model1.state_dict(), 'model1.pth')

    # Evaluation on test data
    model1.eval()
    total_test_loss = 0
    with torch.no_grad():
        for batch, (data, target, mask) in enumerate(test_dataloader):  # Unpacking four elements
            data = data.to(device)
            target = target.to(device)
            mask = mask.to(device)
            
            output = model(data)
            loss = loss_fn(output, target, mask)
            total_test_loss += loss.item()

        print(f"Fold {fold + 1} Test Loss: {total_test_loss / len(test_dataloader)}")


Fold 1


DeferredCudaCallError: CUDA call failed lazily at initialization with error: isinstance() arg 2 must be a type, a tuple of types, or a union

CUDA call was originally invoked at:

['  File "/home/mah20012/envs/cse1/lib/python3.10/runpy.py", line 196, in _run_module_as_main\n    return _run_code(code, main_globals, None,\n', '  File "/home/mah20012/envs/cse1/lib/python3.10/runpy.py", line 86, in _run_code\n    exec(code, run_globals)\n', '  File "/home/mah20012/envs/cse1/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>\n    app.launch_new_instance()\n', '  File "/home/mah20012/envs/cse1/lib/python3.10/site-packages/traitlets/config/application.py", line 1046, in launch_instance\n    app.start()\n', '  File "/home/mah20012/envs/cse1/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 736, in start\n    self.io_loop.start()\n', '  File "/home/mah20012/envs/cse1/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start\n    self.asyncio_l

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Hyperparameters
ntokens = len(base_to_int)  # Vocabulary size
emsize = 300  # Increase embedding dimensions to 300
nhid = 300   # Increase hidden layer size to 300
nlayers = 2  # Increase the number of layers in the transformer to 6
nhead = 100  # Increase the number of heads in multi-head attention to 6
dropout = 0.6  # Increase dropout value to 0.6
weight_decay = 0.0001

# Data loading and preprocessing
max_length = max(df_DMS['sequence'].apply(len))
dataset = RNADataset(df_DMS, max_length)

# Setting up cross-validation
kf = KFold(n_splits=3)


In [ ]:
for fold, (train_index, test_index) in enumerate(kf.split(dataset)):
    print(f"Fold {fold + 1}")
    train_dataset = torch.utils.data.Subset(dataset, train_index)
    test_dataset = torch.utils.data.Subset(dataset, test_index)
    
    train_dataloader = DataLoader(train_dataset, batch_size=160, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=160, shuffle=False)

    # Initialize model and loss function
    model2 = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
    loss_fn = MaskedHuberLoss()
    optimizer = optim.Adam(model.parameters(), weight_decay=0.0001, lr=5e-3)
    
    # Setting up the scheduler
    def lambda_epoch(epoch):
        max_epoch = 3
        return math.pow((1-epoch/max_epoch), 0.9)

    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch)
    
    # Training loop
    num_epochs = 3
    
    for epoch in range(num_epochs):
        start_time = time.time()
        model2.train()
        total_loss = 0.0
        for batch, (seq, target, mask) in enumerate(train_dataloader):
            seq = seq.to(device)
            target = target.to(device)
            mask = mask.to(device)
            
            optimizer.zero_grad()
            output = model2(seq)
            loss = loss_fn(output, target, mask)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            

            if batch % 300 == 0 and batch > 0:
                current = batch * len(seq)
                total = len(train_dataloader.dataset)
                elapsed = time.time() 
                loss_per_batch = total_loss / batch
                print(f'| Epoch {epoch+1:3d}/{num_epochs:3d} | {current:5d}/{total:5d} sequences | Mean Loss Batch: {loss_per_batch} | Elapsed time {elapsed - start_time:.2f}s')
        
        # Execute learning rate scheduler once
        scheduler.step()
        
        elapsed_time = time.time() - start_time
        print(f'Epoch {epoch+1:3d} completed in {elapsed_time:.2f}s, Total loss {total_loss:.4f}')
            
    torch.save(model2.state_dict(), 'model2.pth')

    # Evaluation on test data
    model.eval()
    total_test_loss = 0
    with torch.no_grad():
        for batch, (data, target, mask) in enumerate(test_dataloader):  # Unpacking four elements
            data = data.to(device)
            target = target.to(device)
            mask = mask.to(device)
            
            output = model(data)
            loss = loss_fn(output, target, mask)
            total_test_loss += loss.item()

        print(f"Fold {fold + 1} Test Loss: {total_test_loss / len(test_dataloader)}")


In [ ]:
torch.cuda.empty_cache()

# Inference

In [ ]:
# Loading data
df_test = pd.read_csv('../test_sequences.csv')


In [ ]:
df_test.head()

In [22]:
class RNADataset_Inference(Dataset):
    def __init__(self, dataframe, max_length, context_size=3):
        self.dataframe = dataframe
        self.max_length = max_length
        self.context_size = context_size
        self.pad_id = base_to_int['PAD']

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        sequence = self.dataframe.iloc[idx, 3]

        padded_sequence = self.encode_seq(sequence)

        return padded_sequence
    
    def encode_seq(self, sequence):
        # Convert sequence to list of integers using the dictionary
        encoded_sequence = [base_to_int[base] for base in sequence]
        
        # Pad the sequence to the specified max_length
        padded_sequence = encoded_sequence + [base_to_int['PAD']] * (self.max_length - len(encoded_sequence))
        padded_sequence = padded_sequence[:self.max_length]

        return torch.tensor(padded_sequence, dtype=torch.long)
    

In [23]:
# torch.load('model1_updated.pth')

In [24]:
# import torch
# print(torch.cuda.is_available())

In [ ]:
# model

In [ ]:
import copy


In [ ]:
max_length = max(df_test['sequence'].apply(len))
dataset_infer = RNADataset_Inference(df_test, max_length)
infer_dataloader = DataLoader(dataset_infer, batch_size=50, shuffle=False)

ntokens = len(base_to_int)  # Size of the vocabulary
emsize = 300  # Dimension of embeddings
nhid = 300    # Size of the hidden layer
nlayers = 2   # Number of layers in the transformer
nhead = 100   # Number of heads in multi-head attention
dropout = 0.6 # Dropout rate

# Initialize the model
# model_2A3 = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout)
# model_2A3.load_state_dict(torch.load('model1.pth'))
model_2A3 = copy.deepcopy(model1)
model_2A3.eval().to(device)

# model_DMS = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout)
# model_DMS.load_state_dict(torch.load('model1_updated.pth'))
model_DMS = copy.deepcopy(model2)
model_DMS.eval().to(device)

In [ ]:
dataset_infer[0]

In [ ]:
batch = next(iter(infer_dataloader))
batch

In [ ]:
# # out_2A3 = model_2A3(seq)#.detach().cpu().numpy()
# model = model_2A3.to(device)
# model(seq[:50])
del model, model1, model2

In [ ]:
# torch.cuda.empty_cache()
# torch.empty_cache()

In [ ]:
# # ids = np.empty(shape = (0,1),dtype=int)
# preds_2A3 = np.empty(shape = (0,1),dtype=np.float32)
# preds_DMS = np.empty(shape = (0,1),dtype=np.float32)

# with torch.no_grad():
    # for seq in tqdm(infer_dataloader):
    # #     print(seq)
    #     seq = seq.to(device)
    #     out_2A3 = model_2A3(seq[:50]).detach().cpu().numpy()
    #     out_DMS = model_DMS(seq[:50]).detach().cpu().numpy()
        
    #     clipped_out_2A3 = np.clip(out_2A3, a_min=0,  a_max=1)
    #     clipped_out_DMS = np.clip(out_DMS, a_min=0, a_max=1)
    
    
    #     preds_2A3 = np.append(preds_2A3, clipped_out_2A3)
    #     preds_DMS = np.append(preds_DMS, clipped_out_DMS)
    #     torch.cuda.empty_cache()

In [ ]:
# submission_df = pd.DataFrame({'id':np.arange(0, len(preds_2A3), 1),"reactivity_2A3_MaP": preds_2A3,"reactivity_DMS_MaP": preds_DMS})
# submission_df.to_csv('submission.csv', index=False)

# submission_df.head(20)

In [ ]:
# start_time = time.time()
# predictions = []

# for sequence in tqdm(df_test['sequence'], desc=f'Processing split {i}'):
#     # Preprocess sequence
#     context_padded_sequence = get_context_padded_sequence(sequence)
#     padded_sequence = context_padded_sequence + [base_to_int['PAD']] * (max_length - len(context_padded_sequence))
#     sequence_tensor = torch.tensor(padded_sequence, dtype=torch.long).unsqueeze(0)

#     # Prediction
#     with torch.no_grad():
#         prediction = model(sequence_tensor)

#     # Get and process prediction results
#     predicted_reactivity = prediction.squeeze(0).tolist()
#     predicted_reactivity_trimmed = predicted_reactivity[:len(sequence)]
#     predictions.append(predicted_reactivity_trimmed)
    
#     elapsed_time = time.time() - start_time
#     estimated_remaining_time = (elapsed_time / (i + 1)) * (num_splits - i - 1)
#     print(f"Completed split {i}. Estimated remaining time: {estimated_remaining_time:.2f} seconds")


    
# submission = pd.DataFrame({'id':np.arange(0, len(concat_preds), 1), 'reactivity_DMS_MaP':concat_preds[:,1], 'reactivity_2A3_MaP':concat_preds[:,0]})
# submission.to_csv('submission.csv', index=False)
# submission.head()